In [1]:


import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Layer, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

 
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model, save_model

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import numpy as np
from sklearn.utils import resample
import random
# Import necessary libraries
from keras.datasets import cifar10
dataset = cifar10.load_data()

config = {
    'heldOutClasses': False,
    'seed': 0,
    'ensembleSize': 3,
}
# Turn it into a binary classification problem by making it frogs or not frogs
(train_images, train_labels), (test_images, test_labels) = dataset

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
# Load the dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Separate frog and not frog images
frog_indices = np.where(train_labels == 6)[0]
not_frog_indices = np.where(train_labels != 6)[0]

if config['heldOutClasses']:
    config['heldOutClasses'] = [0, 1]
else:
    config['heldOutClasses'] = []

held_out_class_indices_train = np.where(np.isin(train_labels, config['heldOutClasses']))[0]    
held_out_class_indices_test = np.where(np.isin(test_labels, config['heldOutClasses']))[0]    

not_frog_indices = np.setdiff1d(not_frog_indices, held_out_class_indices_train)

# Downsample majority (not frog)
not_frog_downsampled = resample(not_frog_indices,
                                replace=False, # sample without replacement
                                n_samples=len(frog_indices), # match minority n
                                random_state=27) # reproducible results

# Combine minority and downsampled majority
downsampled_indices = np.concatenate([frog_indices, not_frog_downsampled])

# Downsampled feature and label sets
train_labels = np.where(train_labels == 6, 1, 0)
test_labels = np.where(test_labels == 6, 1, 0)

train_images = train_images[downsampled_indices]
train_labels = train_labels[downsampled_indices]

test_images_held_out = test_images[held_out_class_indices_test]
test_labels_held_out = test_labels[held_out_class_indices_test]

test_images = np.delete(test_images, held_out_class_indices_test, axis=0)
test_labels = np.delete(test_labels, held_out_class_indices_test, axis=0)


# Split the training set into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=config['seed'])

MODEL_FILE = './models/2024-04-30/0'

def _create_model():
    # Input layer
    inputs = Input(shape=(32,32,3))

    # Convolutional and MaxPooling layers
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)

    # Flatten layer
    x = Flatten()(x)

    # Dense layers with dropout
    x = Dense(64, activation='relu')(x)
    x = Dense(16, activation='relu')(x)

    # Output layer
    outputs = Dense(1, activation='sigmoid')(x)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

from tensorflow.keras.layers import Layer



In [9]:
class DNNEnsemble(tf.keras.layers.Layer):
    def __init__(self, models=[], **kwargs):
        super(DNNEnsemble, self).__init__(**kwargs)
        self.models = models

    def call(self, inputs):
        outputs = [model(inputs) for model in self.models]
        return tf.reduce_mean(tf.stack(outputs, axis=0), axis=0)


# Create an ensemble of models
model_list = [load_model(f"model_{i}.h5") for i in range(config['ensembleSize'])]

# Create the DNNEnsemble layer with input shape specified
ensemble = DNNEnsemble(models=model_list)

# Define the rest of the model
inputs = tf.keras.Input(shape=(32, 32, 3))
x = ensemble(inputs)  # Using the ensemble as a layer
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create the functional model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(train_images, train_labels)
print(f"Model accuracy: {accuracy}")


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:



# class OutputAggregationLayer(Layer):
#     def __init__(self):
#         super(OutputAggregationLayer, self).__init__()

#     def call(self, inputs):
#         # Assume inputs is a list of tensors from the ensemble models
#         return tf.reduce_mean(tf.stack(inputs, axis=0), axis=0)

# class DNNEnsemble(Model):
#     def __init__(self, models=[], input_shape=(32, 32, 3)):
#         super(DNNEnsemble, self).__init__()
#         self.models = models
#         self.agg_layer = OutputAggregationLayer()
#         self.build(input_shape)

#     def build(self, input_shape):
#         self.inputs = Input(shape=input_shape)
#         # Collect outputs from each model in a list
#         outputs = [model(self.inputs) for model in self.models]
#         self.outputs = self.agg_layer(outputs)
#         self.model = Model(inputs=self.inputs, outputs=self.outputs)
    
#     def call(self, inputs):
#         outputs = [model(inputs, trainable=False) for model in self.models]
#         return self.agg_layer(outputs)
    
#     def compile(self, optimizer='adam', loss='binary_crossentropy', metrics=[]):
#         super(DNNEnsemble, self).compile(optimizer=optimizer, loss=loss, metrics=metrics)
#         self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

#     def fit(self, *args, **kwargs):
#         return self.model.fit(*args, **kwargs)

#     def evaluate(self, *args, **kwargs):
#         return self.model.evaluate(*args, **kwargs)


# # Example usage
# if __name__ == "__main__":
#     model_list = []

#     def train_and_save_model(i):
#         model = _create_model()  # Ensure _create_model compiles the model
#         # Fit the model
#         model.fit(train_images, train_labels, epochs=5, batch_size=32)
#         # Save the model
#         save_model(model, f"model_{i}.h5")

#     # Example usage:
#     # for i in range(config['ensembleSize']):
#     #     train_and_save_model(i)

#     # Create an ensemble of 3 models
#     model_list = [load_model(f"model_{i}.h5") for i in range(config['ensembleSize'])]

#     ensemble = DNNEnsemble(models=model_list, input_shape=(32, 32, 3))
#     # Compile the ensemble
#     ensemble.compile(metrics=['accuracy'])

#     ensemble.fit(train_images, train_labels, epochs=5, batch_size=32)

#     # Evaluate the ensemble
#     loss, accuracy = ensemble.evaluate(train_images, train_labels)
#     print(f"Ensemble accuracy: {accuracy}")
